<a href="https://colab.research.google.com/github/jenh/epub-ocr-and-translate/blob/master/onmt-helpers/EOAT_OpenNMT_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install dependencies and connect to Google Drive

In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py

In [0]:
!pip install -r OpenNMT-py/requirements.txt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocess

Preprocess files. This command is vanilla, no bpe or sentencepiece or any other special sauce, we're training straight from corpus --- you can customize this as needed.

In [0]:
!python /content/OpenNMT-py/preprocess.py -train_src /content/drive/My\ Drive/my_working_dir/src_corpus -train_tgt /content/drive/My\ Drive/my_working_dir/tgt_corpus -valid_src /content/drive/My\ Drive/my_working_dir/src_val -valid_tgt /content/drive/My\ Drive/my_working_dir/tgt_val -save_data /content/drive/My\ Drive/my_working_dir/goognodyndict --lower --share_vocab --src_seq_length 10000 --tgt_seq_length 10000

# Check GPU

Verify GPU status.

In [0]:
!nvidia-smi


Sat Sep 21 01:11:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Train (First run)

Create and run a training command like the following (yours may differ based on what your aims are --- I've cribbed my settings based on significant Googling, and they are as always subject to change). On Google Colab, you want -world_size 1 (we have one GPU available to us) and -gpu_ranks 0 (its id is 0).

In [0]:
!python /content/OpenNMT-py/train.py -data /content/drive/My\ Drive/my_working_dir/my_training_data -save_model /content/drive/My\ Drive/my_working_dir/my_training_data-model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0  -param_init_glorot -label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 3000 -report_every 1000 -world_size 1 -gpu_ranks 0

# Train (from model)

Once you've been interrupted, restart training with -train_from to start where you left off.

In [0]:
!python /content/OpenNMT-py/train.py -data /content/drive/My\ Drive/my_working_dir/my_training_data -save_model /content/drive/My\ Drive/my_working_dir/my_training_data-model -train_from /content/drive/My\ Drive/my_working_dir/my-last-good-generated-model_step_nnnnn.pt -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000  -max_generator_batches 2 -dropout 0.1 -batch_size 2048 -batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0  -param_init_glorot -label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 3000 -report_every 1000 -world_size 1 -gpu_ranks 0